<center> <h1> <u> <font color='red'> Hands on Training on Bangla Language Processing Collocated with CLBLP'23 </font> </u> </h1> </center>

#Libraries

In [2]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

CPU times: user 3.78 s, sys: 527 ms, total: 4.3 s
Wall time: 8.62 s


## Aggresive Dataset


In [ ]:
!wget -O aggression.zip https://www.dropbox.com/s/8h0knnj1fvv4s3t/aggression.zip?dl=0
     

# extract the zip file and save the files into bangla-digit folder
import zipfile
zip_ref = zipfile.ZipFile("aggression.zip", 'r')
zip_ref.extractall("")
zip_ref.close()

In [46]:
dataset = pd.read_excel("aggression/train-multi.xlsx")
dataset = dataset.iloc[:,1:]
dataset

,cleaned,Class,Label
0,বর্তমানে শেখ পরিবার মানেই আলাদিনের চ্যাড়াগ কি ...,PoAG,1
1,মাগি আগে কথা বলা শেখ তার পর ভিডিও বানাইস,VeAG,2
2,তোর পুটকি দিয়া নল ঢুকাইয়া মুখ দিয়া গুলি করুম,VeAG,2
3,মাদারচোদ তো নিজে কইতাছে যে সে পাগল চোদা,VeAG,2
4,ওরা কোরান নিয়ে যে দাবি করে পুরোটাই ভন্ডামি আর...,ReAG,0
...,...,...,...
5441,এতদিন মুসলমান সরকার ছিল দিল্লিতে এইবার হিন্দু ...,VeAG,2
5442,নারীর ক্ষমতায়ন করা যাবে তবে পর্দা বজায় রেখে যে...,GeAG,3
5443,সে যদি আমার সামনে থাকতো টা হলে ওর মুখ...,VeAG,2
5444,আমি আমার জানামতে বা আমার বিবেকে যাহা উপলব্ধি ক...,ReAG,0


In [ ]:
val_dataset = pd.read_excel("aggression/val-multi.xlsx")
val_dataset

In [ ]:
test_dataset = pd.read_excel("aggression/test-multi.xlsx")
test_dataset

In [27]:
dataset.Class.value_counts().keys()

Index(['ReAG', 'PoAG', 'VeAG', 'GeAG'], dtype='object')

In [31]:
dataset.Class.value_counts()

ReAG    1794
PoAG    1655
VeAG    1629
GeAG     368
Name: Class, dtype: int64

##Evaluation Metrics

In [70]:
from sklearn.metrics import confusion_matrix,classification_report 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
'''Evaluation Parameters'''

def print_metrices(true,pred):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred,target_names=['ReAG', 'PoAG', 'VeAG', 'GeAG']))
    print("Accuracy : ",accuracy_score(true,pred))
    print("Precison : ",precision_score(true,pred, average = 'weighted'))
    print("Recall : ",recall_score(true,pred,  average = 'weighted'))
    print("F1 : ",f1_score(true,pred,  average = 'weighted'))

In [ ]:
pip install ktrain

In [51]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [52]:
import ktrain
from ktrain import text

In [58]:
X_train = dataset['cleaned'].tolist()
y_train = dataset['Label'].tolist()

X_valid = val_dataset['cleaned'].tolist()
y_valid = val_dataset['Label'].tolist()

X_test = test_dataset['cleaned'].tolist()
#y_test = test_data['label'].tolist()

In [54]:
categories = dataset.Class.value_counts().keys()
categories = categories.tolist()
categories

['ReAG', 'PoAG', 'VeAG', 'GeAG']

In [55]:
model_name = 'bert-base-multilingual-cased'
trans = text.Transformer(model_name,maxlen=50,class_names=categories)

In [59]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)
#test = trans.preprocess_test(X_test,y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 14
	95percentile : 36
	99percentile : 56


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 14
	95percentile : 35
	99percentile : 48


In [60]:
model = trans.get_classifier()

In [61]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=12)

In [62]:
learner.fit_onecycle(2e-5,5)     #2e-5        



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
454/454 [==============================] - 134s 200ms/step - loss: 0.8014 - accuracy: 0.7211 - val_loss: 0.5197 - val_accuracy: 0.8282
Epoch 2/5
454/454 [==============================] - 86s 190ms/step - loss: 0.4382 - accuracy: 0.8548 - val_loss: 0.4237 - val_accuracy: 0.8634
Epoch 3/5
454/454 [==============================] - 87s 191ms/step - loss: 0.3476 - accuracy: 0.8871 - val_loss: 0.4183 - val_accuracy: 0.8590
Epoch 4/5
454/454 [==============================] - 92s 203ms/step - loss: 0.2366 - accuracy: 0.9236 - val_loss: 0.3722 - val_accuracy: 0.8811
Epoch 5/5
454/454 [==============================] - 88s 193ms/step - loss: 0.1447 - accuracy: 0.9572 - val_loss: 0.3836 - val_accuracy: 0.8737


In [63]:
learner.validate(class_names=categories)

22/22 [==============================] - 6s 124ms/step
              precision    recall  f1-score   support

        ReAG       0.90      0.91      0.90       210
        PoAG       0.93      0.95      0.94       229
        VeAG       0.82      0.87      0.84       194
        GeAG       0.66      0.40      0.49        48

    accuracy                           0.87       681
   macro avg       0.82      0.78      0.79       681
weighted avg       0.87      0.87      0.87       681



array([[191,   5,  13,   1],
       [  3, 217,   9,   0],
       [ 12,   5, 168,   9],
       [  7,   6,  16,  19]])

##Save and Load Model

In [64]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [65]:
# Save and load the model
predictor.save('aggressive-bert')

In [66]:
predictor = ktrain.load_predictor('aggressive-bert')

In [ ]:
y_pred = predictor.predict(X_test)
y_pred

In [71]:
print_metrices(test_dataset['Class'],y_pred)

[[ 23   5   6  12]
 [  2 189   5   5]
 [  2   7 194  10]
 [ 10   4   8 198]]
              precision    recall  f1-score   support

        ReAG       0.62      0.50      0.55        46
        PoAG       0.92      0.94      0.93       201
        VeAG       0.91      0.91      0.91       213
        GeAG       0.88      0.90      0.89       220

    accuracy                           0.89       680
   macro avg       0.83      0.81      0.82       680
weighted avg       0.88      0.89      0.89       680

Accuracy :  0.888235294117647
Precison :  0.884568808406685
Recall :  0.888235294117647
F1 :  0.8858929232896672


##Reference


1. [Text Classification Using Transformers](https://nbviewer.jupyter.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-A3-hugging_face_transformers.ipynb)

2. [Related Article](https://towardsdatascience.com/text-classification-with-hugging-face-transformers-in-tensorflow-2-without-tears-ee50e4f3e7ed)

3. [Ktrain](https://github.com/amaiya/ktrain)

4. [m-BERT](https://huggingface.co/bert-base-multilingual-cased)